# [Problem 1] Cross Validation

In [1]:
# importing the necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Loading the dataset
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')
train.shape, test.shape

((307511, 122), (48744, 121))

In [3]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [5]:
train.isnull().sum().sum()

9152465

In [6]:
pd.set_option('display.max_row', 122)
pd.set_option('display.max_column', 122)

In [7]:
train_null = train.isnull().sum()[train.isnull().sum()!=0].sort_values(ascending=False)
train_null

COMMONAREA_MEDI                 214865
COMMONAREA_AVG                  214865
COMMONAREA_MODE                 214865
NONLIVINGAPARTMENTS_AVG         213514
NONLIVINGAPARTMENTS_MODE        213514
NONLIVINGAPARTMENTS_MEDI        213514
FONDKAPREMONT_MODE              210295
LIVINGAPARTMENTS_AVG            210199
LIVINGAPARTMENTS_MEDI           210199
LIVINGAPARTMENTS_MODE           210199
FLOORSMIN_AVG                   208642
FLOORSMIN_MEDI                  208642
FLOORSMIN_MODE                  208642
YEARS_BUILD_AVG                 204488
YEARS_BUILD_MEDI                204488
YEARS_BUILD_MODE                204488
OWN_CAR_AGE                     202929
LANDAREA_AVG                    182590
LANDAREA_MEDI                   182590
LANDAREA_MODE                   182590
BASEMENTAREA_MODE               179943
BASEMENTAREA_AVG                179943
BASEMENTAREA_MEDI               179943
EXT_SOURCE_1                    173378
NONLIVINGAREA_AVG               169682
NONLIVINGAREA_MODE       

In [8]:
# Percentage of missing values
percent_missing = 100 * train_null / len(train)
        
# Make a table with the results
missing_val_table = pd.concat([train_null, percent_missing], axis=1)
        
# Rename the columns
missing_val_table = missing_val_table.rename(
  columns = {0 : 'Missing Values', 1 : 'percentage'}).round(1)
missing_val_table

,Missing Values,percentage
COMMONAREA_MEDI,214865,69.9
COMMONAREA_AVG,214865,69.9
COMMONAREA_MODE,214865,69.9
NONLIVINGAPARTMENTS_AVG,213514,69.4
NONLIVINGAPARTMENTS_MODE,213514,69.4
NONLIVINGAPARTMENTS_MEDI,213514,69.4
FONDKAPREMONT_MODE,210295,68.4
LIVINGAPARTMENTS_AVG,210199,68.4
LIVINGAPARTMENTS_MEDI,210199,68.4
LIVINGAPARTMENTS_MODE,210199,68.4


In [9]:
# Number of each type of column
train.dtypes.value_counts()
# object columns contain strings and are categorical features. 

float64    65
int64      41
object     16
dtype: int64

In [10]:
# Selceting the object features
train_object = train.select_dtypes('object')
train_object

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,"Stone, brick",No
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,block of flats,Block,No
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government,NaN,NaN,NaN,NaN
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,THURSDAY,Services,reg oper account,block of flats,"Stone, brick",No
307507,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,NaN,MONDAY,XNA,reg oper account,block of flats,"Stone, brick",No
307508,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,THURSDAY,School,reg oper account,block of flats,Panel,No
307509,Cash loans,F,N,Y,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 1,NaN,block of flats,"Stone, brick",No


In [11]:
# Number of unique classes in each object column
train_object.apply(pd.Series.nunique, axis = 0).sort_values(ascending=False)

ORGANIZATION_TYPE             58
OCCUPATION_TYPE               18
NAME_INCOME_TYPE               8
WALLSMATERIAL_MODE             7
WEEKDAY_APPR_PROCESS_START     7
NAME_TYPE_SUITE                7
NAME_HOUSING_TYPE              6
NAME_FAMILY_STATUS             6
NAME_EDUCATION_TYPE            5
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
CODE_GENDER                    3
EMERGENCYSTATE_MODE            2
FLAG_OWN_REALTY                2
FLAG_OWN_CAR                   2
NAME_CONTRACT_TYPE             2
dtype: int64

In [12]:
# object columns with missing values
missin_obj = train_object.isnull().sum()
missin_obj = missin_obj[missin_obj!=0].sort_values(ascending=False)
missin_obj

FONDKAPREMONT_MODE     210295
WALLSMATERIAL_MODE     156341
HOUSETYPE_MODE         154297
EMERGENCYSTATE_MODE    145755
OCCUPATION_TYPE         96391
NAME_TYPE_SUITE          1292
dtype: int64

In [13]:
missin_obj = missin_obj.index
# Code to get number of categories in missing value columns
print("Number of Categories in: ")
for ColName in missin_obj:
    print("{} = {}".format(ColName, len(train[ColName].unique())))

Number of Categories in: 
FONDKAPREMONT_MODE = 5
WALLSMATERIAL_MODE = 8
HOUSETYPE_MODE = 4
EMERGENCYSTATE_MODE = 3
OCCUPATION_TYPE = 19
NAME_TYPE_SUITE = 8


In [14]:
# Code to get most_frequent_category in missing value columns
print("most_frequent_category in: ")
for ColName in missin_obj:
    print("{} = {}".format(ColName, train[ColName].mode()[0]))

most_frequent_category in: 
FONDKAPREMONT_MODE = reg oper account
WALLSMATERIAL_MODE = Panel
HOUSETYPE_MODE = block of flats
EMERGENCYSTATE_MODE = No
OCCUPATION_TYPE = Laborers
NAME_TYPE_SUITE = Unaccompanied


In [15]:
# Function to impute most occured category and add importance vairable
def impute_nan(Data,ColName):
    Mode_Category = Data[ColName].mode()[0]
    
    ## 2.1 Replace NAN values with most occured category in actual vairable
    
    Data[ColName].fillna(Mode_Category,inplace=True)
# Call function to impute NAN values and add new importance feature
for Columns in missin_obj:
    impute_nan(train, Columns)
train[missin_obj]

,FONDKAPREMONT_MODE,WALLSMATERIAL_MODE,HOUSETYPE_MODE,EMERGENCYSTATE_MODE,OCCUPATION_TYPE,NAME_TYPE_SUITE
0,reg oper account,"Stone, brick",block of flats,No,Laborers,Unaccompanied
1,reg oper account,Block,block of flats,No,Core staff,Family
2,reg oper account,Panel,block of flats,No,Laborers,Unaccompanied
3,reg oper account,Panel,block of flats,No,Laborers,Unaccompanied
4,reg oper account,Panel,block of flats,No,Core staff,Unaccompanied
...,...,...,...,...,...,...
307506,reg oper account,"Stone, brick",block of flats,No,Sales staff,Unaccompanied
307507,reg oper account,"Stone, brick",block of flats,No,Laborers,Unaccompanied
307508,reg oper account,Panel,block of flats,No,Managers,Unaccompanied
307509,reg oper account,"Stone, brick",block of flats,No,Laborers,Unaccompanied


In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [17]:
le = LabelEncoder()

# Iterate through the columns
for col in train:
    if train[col].dtype == 'object':
        le.fit(train[col])
            # Transform both training and testing data
        train[col] = le.transform(train[col])
        

In [18]:
train[missin_obj]

,FONDKAPREMONT_MODE,WALLSMATERIAL_MODE,HOUSETYPE_MODE,EMERGENCYSTATE_MODE,OCCUPATION_TYPE,NAME_TYPE_SUITE
0,2,5,0,0,8,6
1,2,0,0,0,3,1
2,2,4,0,0,8,6
3,2,4,0,0,8,6
4,2,4,0,0,3,6
...,...,...,...,...,...,...
307506,2,5,0,0,14,6
307507,2,5,0,0,8,6
307508,2,4,0,0,10,6
307509,2,5,0,0,8,6


### Dealing with anomalies

In [19]:
train["DAYS_EMPLOYED"].describe()

count    307511.000000
mean      63815.045904
std      141275.766519
min      -17912.000000
25%       -2760.000000
50%       -1213.000000
75%        -289.000000
max      365243.000000
Name: DAYS_EMPLOYED, dtype: float64

In [20]:
train["DAYS_EMPLOYED"]

0           -637
1          -1188
2           -225
3          -3039
4          -3038
           ...  
307506      -236
307507    365243
307508     -7921
307509     -4786
307510     -1262
Name: DAYS_EMPLOYED, Length: 307511, dtype: int64

In [21]:
train["DAYS_EMPLOYED"]/365

0           -1.745205
1           -3.254795
2           -0.616438
3           -8.326027
4           -8.323288
             ...     
307506      -0.646575
307507    1000.665753
307508     -21.701370
307509     -13.112329
307510      -3.457534
Name: DAYS_EMPLOYED, Length: 307511, dtype: float64

In [22]:
# Create an anomalous flag column
train['DAYS_EMPLOYED_ANOM'] = train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [23]:
# Desplaying columns with negitive values
train.loc[:,["DAYS_EMPLOYED", 'DAYS_BIRTH', 'DAYS_REGISTRATION','DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE'] ]

,DAYS_EMPLOYED,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE
0,-637.0,-9461,-3648.0,-2120,-1134.0
1,-1188.0,-16765,-1186.0,-291,-828.0
2,-225.0,-19046,-4260.0,-2531,-815.0
3,-3039.0,-19005,-9833.0,-2437,-617.0
4,-3038.0,-19932,-4311.0,-3458,-1106.0
...,...,...,...,...,...
307506,-236.0,-9327,-8456.0,-1982,-273.0
307507,NaN,-20775,-4388.0,-4090,0.0
307508,-7921.0,-14966,-6737.0,-5150,-1909.0
307509,-4786.0,-11961,-2562.0,-931,-322.0


In [24]:
train[['DAYS_BIRTH']] = train[['DAYS_BIRTH']]/-365
train[['DAYS_REGISTRATION']] = train[['DAYS_REGISTRATION']]/-365
train[['DAYS_ID_PUBLISH']] = train[['DAYS_ID_PUBLISH']] / -365
train[['DAYS_LAST_PHONE_CHANGE']] = train[['DAYS_LAST_PHONE_CHANGE']] / -365
train['DAYS_EMPLOYED'] = train["DAYS_EMPLOYED"]/-365

In [25]:
train.loc[:,["DAYS_EMPLOYED", 'DAYS_BIRTH', 'DAYS_REGISTRATION','DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE'] ]

,DAYS_EMPLOYED,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE
0,1.745205,25.920548,9.994521,5.808219,3.106849
1,3.254795,45.931507,3.249315,0.797260,2.268493
2,0.616438,52.180822,11.671233,6.934247,2.232877
3,8.326027,52.068493,26.939726,6.676712,1.690411
4,8.323288,54.608219,11.810959,9.473973,3.030137
...,...,...,...,...,...
307506,0.646575,25.553425,23.167123,5.430137,0.747945
307507,NaN,56.917808,12.021918,11.205479,-0.000000
307508,21.701370,41.002740,18.457534,14.109589,5.230137
307509,13.112329,32.769863,7.019178,2.550685,0.882192


In [26]:
#selecting numerical features
train_num = train.select_dtypes('number')
train_num

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,6,7,4,3,1,0.018801,25.920548,1.745205,9.994521,5.808219,NaN,1,1,0,1,1,0,8,1.0,2,2,6,10,0,0,0,0,0,0,5,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.0220,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,2,0,0.0149,5,0,2.0,2.0,2.0,2.0,3.106849,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,1,4,1,1,1,0.003541,45.931507,3.254795,3.249315,0.797260,NaN,1,1,0,1,1,0,3,2.0,1,1,1,11,0,0,0,0,0,0,39,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.0790,0.0554,0.0,0.0000,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,2,0,0.0714,0,0,1.0,0.0,1.0,0.0,2.268493,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,1,1,1,0,67500.0,135000.0,6750.0,135000.0,6,7,4,3,1,0.010032,52.180822,0.616438,11.671233,6.934247,26.0,1,1,1,1,1,0,8,1.0,2,2,1,9,0,0,0,0,0,0,11,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,NaN,4,0,0.0,0.0,0.0,0.0,2.232877,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,6,7,4,0,1,0.008019,52.068493,8.326027,26.939726,6.676712,NaN,1,1,0,1,0,0,8,2.0,2,2,6,17,0,0,0,0,0,0,5,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [27]:
# selecting null columns from train_Num
train_num_null = pd.DataFrame(train_num.isnull().sum()[train.isnull().sum()!=0].sort_values(ascending=False))
train_num_null

,0
COMMONAREA_MEDI,214865
COMMONAREA_AVG,214865
COMMONAREA_MODE,214865
NONLIVINGAPARTMENTS_MODE,213514
NONLIVINGAPARTMENTS_AVG,213514
NONLIVINGAPARTMENTS_MEDI,213514
LIVINGAPARTMENTS_AVG,210199
LIVINGAPARTMENTS_MODE,210199
LIVINGAPARTMENTS_MEDI,210199
FLOORSMIN_MODE,208642


In [28]:
num_null_column = train_num.columns[train_num.isnull().any()]
num_null_column = num_null_column.values.reshape(-1, 1)

In [29]:
for i in num_null_column:
    # Imputation
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    train[i] = imputer.fit_transform(train[i])

In [30]:
train = train.drop(['DAYS_EMPLOYED_ANOM'], axis=1)

In [31]:
train.isnull().any().any()

False

In [32]:
 train.shape, test.shape

((307511, 122), (48744, 121))

In [33]:
threshold = 0.8

In [34]:
# find and remove correlated features
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr            

In [35]:
corr_features = correlation(train, threshold)
#corr_features = correlation(train.iloc[:,:-1], threshold)
len(set(corr_features))
corr_features

{'AMT_GOODS_PRICE',
 'APARTMENTS_MEDI',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MEDI',
 'BASEMENTAREA_MODE',
 'CNT_FAM_MEMBERS',
 'COMMONAREA_MEDI',
 'COMMONAREA_MODE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'ELEVATORS_AVG',
 'ELEVATORS_MEDI',
 'ELEVATORS_MODE',
 'ENTRANCES_MEDI',
 'ENTRANCES_MODE',
 'FLOORSMAX_MEDI',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MEDI',
 'FLOORSMIN_MODE',
 'LANDAREA_MEDI',
 'LANDAREA_MODE',
 'LIVE_CITY_NOT_WORK_CITY',
 'LIVE_REGION_NOT_WORK_REGION',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_AVG',
 'LIVINGAREA_MEDI',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MEDI',
 'NONLIVINGAREA_MODE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'REGION_RATING_CLIENT_W_CITY',
 'TOTALAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MEDI',
 'YEARS_BUILD_MODE'}

In [36]:
train = train.drop(corr_features, axis=1)
test = test.drop(corr_features, axis=1)

In [37]:
 train.shape, test.shape

((307511, 84), (48744, 83))

In [38]:
# spliting the train datset
Xtrain = train.drop(['TARGET'], axis=1)
y = train[['TARGET']]

Xtrain, y = Xtrain.align(y, join='inner', axis=0)


In [39]:
X = Xtrain.to_numpy()
y = y.to_numpy()

In [40]:
X.shape, y.shape

((307511, 83), (307511, 1))

In [41]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [42]:
# prepare the cross-validation procedure
cv = KFold(n_splits=2, random_state=None, shuffle=False)

In [43]:
for train_index, test_index in cv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [153756 153757 153758 ... 307508 307509 307510] TEST: [     0      1      2 ... 153753 153754 153755]
TRAIN: [     0      1      2 ... 153753 153754 153755] TEST: [153756 153757 153758 ... 307508 307509 307510]


### [Problem 2] Grid search

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [45]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import roc_auc_score
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [47]:
from sklearn.metrics import accuracy_score
models = ['RFC', 'LogisticRegression']

clfs = [RandomForestClassifier(random_state=1, n_jobs=-1),
        LogisticRegression()
#         SVC(random_state=1, probability=True)
       ]
    
        
params = {
          models[0]:{'n_estimators':[1,5,10,100], 'criterion':['entropy','gini'],
                     'min_samples_split':[2], 'min_samples_leaf':[1, 4]},
          
          models[1]:{'C':[1,5,10,100], 'tol':[0.0001,0.005], 'solver': ['lbfgs', 'liblinear']}
          
#           models[2]:{'C':[1.0, 2000], 'tol':[0.001, 0.005], 'max_iter':[-1],
#                      'kernel':['sigmoid', 'linear', 'rbf'],
#                      'gamma':['scale', 'auto'], 'verbose':[True, False]},
         }

test_score = [] 
for name, estimator in zip( models, clfs):
    print(name)
    model = GridSearchCV(estimator, params[name], scoring='accuracy', refit='True', n_jobs= -1, cv=2)
    model.fit(X_train, y_train)
    print('complete Training')
        
    print("best params: " + str(model.best_params_))
    print("best score: " + str(model.best_score_))
    print("best estimator: " + str(model.best_estimator_))
        
    
    accuracy = accuracy_score(y_test, model.predict(X_test))
    print("Accuracy: {:.4%}".format(accuracy))
    
    test_score.append((name, accuracy,model.best_score_, model.best_estimator_))
score = pd.DataFrame(test_score,columns=['model', 'Accuracy', 'best_score', 'Estimator'])         
print(score)

RFC
complete Training
best params: {'criterion': 'entropy', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 1}
best score: 0.8060628528317594
best estimator: RandomForestClassifier(criterion='entropy', min_samples_leaf=4, n_estimators=1,
                       n_jobs=-1, random_state=1)
Accuracy: 87.0547%
LogisticRegression
complete Training
best params: {'C': 5, 'solver': 'liblinear', 'tol': 0.0001}
best score: 0.9182991232862457
best estimator: LogisticRegression(C=5, solver='liblinear')
Accuracy: 91.9944%
                model  Accuracy  best_score  \
0                 RFC  0.870547    0.806063   
1  LogisticRegression  0.919944    0.918299   

                                           Estimator  
0  (DecisionTreeClassifier(criterion='entropy', m...  
1        LogisticRegression(C=5, solver='liblinear')  


# [Problem 3] Survey from Kaggle Notebooks

1. K-Fold Cross Validation
2. Features selection
3. K-Fold cross validation
4. Overview of different categorical treatments
5. Automated Hyperparameter Tuning
6. Dealing with anomalies

# [Problem 4] Creating a model with high generalization performance

In [48]:
import lightgbm as lgb

# creating an instance of the model
cls = lgb.LGBMClassifier()

# save the default params
default_params = cls.get_params()

# number of folds
N_FOLDS = 5

# creating a dataset
train_set = lgb.Dataset(data = X_train)

# Cross validation results when avoid overfitting
cv_results = lgb.cv(default_params, train_set, num_boost_round = 2, early_stopping_rounds = 1, metrics = 'auc', nfold = N_FOLDS, seed = 42)

# displaying the results
print('The maximum validation ROC AUC was: {:.5f}.'.format(cv_results['auc-mean'][-1]))
print('The optimal number of boosting rounds (estimators) was {}.'.format(len(cv_results['auc-mean'])))

[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.081425 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5930
[LightGBM] [Info] Number of data points in the train set: 123004, number of used features: 78
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: importance_type
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Auto

1. The whole dataset was imported 
2. Subset of the dataset were created for numbers and object variable
3. The dataset was split using kfold
4. GridsearchCV was to find the best model and params to fine tune my classiffiers
5. After finding the results i tested the LGBM classifier and checked it ROC which is good

# [Problem 5] Final model selection

In [50]:
# Selceting the object features
test_object = test.select_dtypes('object')
test_object

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
0,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Married,House / apartment,NaN,TUESDAY,Kindergarten,NaN,block of flats,"Stone, brick",No
1,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Low-skill Laborers,FRIDAY,Self-employed,NaN,NaN,NaN,NaN
2,Cash loans,M,Y,Y,NaN,Working,Higher education,Married,House / apartment,Drivers,MONDAY,Transport: type 3,NaN,NaN,NaN,NaN
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Sales staff,WEDNESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
4,Cash loans,M,Y,N,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,NaN,FRIDAY,Business Entity Type 3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Widow,House / apartment,NaN,WEDNESDAY,Other,NaN,NaN,NaN,NaN
48740,Cash loans,F,N,N,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,Sales staff,MONDAY,Trade: type 7,NaN,NaN,NaN,NaN
48741,Cash loans,F,Y,Y,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,NaN,WEDNESDAY,Business Entity Type 3,NaN,block of flats,"Stone, brick",No
48742,Cash loans,M,N,N,Family,Commercial associate,Higher education,Married,House / apartment,Managers,MONDAY,Self-employed,NaN,block of flats,Panel,No


In [51]:
# object columns with missing values
testNull_obj = test_object.isnull().sum()
testNull_obj = testNull_obj[testNull_obj!=0].sort_values(ascending=False)
testNull_obj

FONDKAPREMONT_MODE     32797
WALLSMATERIAL_MODE     23893
HOUSETYPE_MODE         23619
EMERGENCYSTATE_MODE    22209
OCCUPATION_TYPE        15605
NAME_TYPE_SUITE          911
dtype: int64

In [52]:
testNull_obj = testNull_obj.index

In [53]:
# def impute_nan(Data,ColName):
#     Mode_Category = test[ColName].mode()[0]
    
#     ## 2.1 Replace NAN values with most occured category in actual vairable
    
#     test[ColName].fillna(Mode_Category,inplace=True)

for Columns in testNull_obj:
    impute_nan(test, Columns)
test[testNull_obj]

,FONDKAPREMONT_MODE,WALLSMATERIAL_MODE,HOUSETYPE_MODE,EMERGENCYSTATE_MODE,OCCUPATION_TYPE,NAME_TYPE_SUITE
0,reg oper account,"Stone, brick",block of flats,No,Laborers,Unaccompanied
1,reg oper account,Panel,block of flats,No,Low-skill Laborers,Unaccompanied
2,reg oper account,Panel,block of flats,No,Drivers,Unaccompanied
3,reg oper account,Panel,block of flats,No,Sales staff,Unaccompanied
4,reg oper account,Panel,block of flats,No,Laborers,Unaccompanied
...,...,...,...,...,...,...
48739,reg oper account,Panel,block of flats,No,Laborers,Unaccompanied
48740,reg oper account,Panel,block of flats,No,Sales staff,Unaccompanied
48741,reg oper account,"Stone, brick",block of flats,No,Laborers,Unaccompanied
48742,reg oper account,Panel,block of flats,No,Managers,Family


In [54]:
# Iterate through the columns
for col in test:
    if test[col].dtype == 'object':
        le.fit(test[col])
            # Transform both training and testing data
        test[col] = le.transform(test[col])
test[testNull_obj]

,FONDKAPREMONT_MODE,WALLSMATERIAL_MODE,HOUSETYPE_MODE,EMERGENCYSTATE_MODE,OCCUPATION_TYPE,NAME_TYPE_SUITE
0,2,5,0,0,8,6
1,2,4,0,0,9,6
2,2,4,0,0,4,6
3,2,4,0,0,14,6
4,2,4,0,0,8,6
...,...,...,...,...,...,...
48739,2,4,0,0,8,6
48740,2,4,0,0,14,6
48741,2,5,0,0,8,6
48742,2,4,0,0,10,1


In [55]:
#selecting numerical features
test_num = test.select_dtypes('number')
test_num

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,REGION_RATING_CLIENT,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,0,0,0,1,0,135000.0,568800.0,20560.5,6,6,1,1,1,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,8,2,5,18,0,0,0,0,28,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,NaN,NaN,2,0,5,0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,0,1,0,1,0,99000.0,222768.0,17370.0,6,6,4,1,1,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,9,2,0,9,0,0,0,0,42,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,4,0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,0,1,1,1,0,202500.0,663264.0,69777.0,6,6,1,1,1,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,4,2,1,14,0,0,0,0,54,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,4,0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,0,0,0,1,2,315000.0,1575000.0,49018.5,6,6,4,1,1,0.026392,-13976,-1866,-2000.0,-4208,NaN,1,1,0,1,1,0,14,2,6,11,0,0,0,0,5,0.525734,0.509677,0.612704,0.3052,0.1974,0.9970,0.9592,0.1165,0.2759,0.3750,0.0417,0.2042,0.2404,0.0386,0.0800,2,0,4,0,0.0,0.0,-1805.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,0,1,1,0,1,180000.0,625500.0,32067.0,6,6,4,1,1,0.010032,-13040,-2191,-4000.0,-4262,16.0,1,1,1,1,0,0,8,2,0,5,0,0,0,1,5,0.202145,0.425687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,4,0,0.0,0.0,-821.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0,0,0,1,0,121500.0,412560.0,17473.5,6,6,4,4,1,0.002042,-19970,-5169,-9094.0,-3399,NaN,1,1,1,1,1,0,8,3,6,16,0,0,0,0,33,NaN,0.648575,0.643026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,4,0,1.0,0.0,-684.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,0,0,0,0,2,157500.0,622413.0,31909.5,6,1,4,1,1,0.035792,-11186,-1149,-3015.0,-3003,NaN,1,1,0,1,0,0,14,2,1,11,0,0,0,1,51,NaN,0.684596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,4,0,2.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,0,0,1,1,1,202500.0,315000.0,33205.5,6,1,4,1,1,0.026392,-15922,-3037,-2681.0,-1504,4.0,1,1,0,1,1,0,8,2,6,12,0,0,0,0,

In [56]:
num_null_column = test_num.columns[test_num.isnull().any()]
num_null_column = num_null_column.values.reshape(-1, 1)

In [57]:
for i in num_null_column:
    # Imputation
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    test[i] = imputer.fit_transform(test[i])

In [58]:
test.isnull().any().any()

False

In [59]:
# standardizing the data
scaler = MinMaxScaler()
X_test_trans = scaler.fit_transform(test)

# predicting
model_pred = model.predict(X_test_trans)

In [60]:
sub = test[['SK_ID_CURR']]
sub['TARGET'] = model_pred
submit_3 = sub

submit_3.to_csv("submit_3.csv", index=False)

In [61]:
submit_3

,SK_ID_CURR,TARGET
0,100001,0
1,100005,0
2,100013,0
3,100028,0
4,100038,0
...,...,...
48739,456221,0
48740,456222,0
48741,456223,0
48742,456224,0


<img src="submit_3.JPG"/>